In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
gdrive	sample_data


In [2]:
import os

home_dir = '/content/gdrive/My Drive/AYield/China'
os.chdir(home_dir)
!pwd
# !ls
# !pip3 install --upgrade IPython
!pip install fire
!pip install geopandas
!pip install geojson
!pip install tqdm

from tqdm import tqdm
import geojson

from glob import glob

# sudo apt-get update & apt-get upgrade

# !pip install --upgrade google-api-python-client
# !gcloud init 

/content/gdrive/My Drive/AYield/China
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# !git clone https://github.com/gabrieltseng/pycrop-yield-prediction.git

In [4]:
code_dir = os.path.join(home_dir, 'pycrop-yield-prediction')
os.chdir(code_dir)
!pwd
!ls

/content/gdrive/My Drive/AYield/China/pycrop-yield-prediction
cyp   diagrams	       LICENSE	  run2.py  run.py
data  environment.yml  README.md  run3.py


In [5]:
# fn
# !earthengine authenticate
# !earthengine task cancel all

In [6]:
# files = glob(os.path.join(code_dir, 'data', 'crop_yield-data_image', '*.tif'))
# for ifile in tqdm(files):
#   os.remove(ifile)

# files = glob(os.path.join(code_dir, 'data', 'crop_yield-data_mask', '*.tif'))
# for ifile in tqdm(filezzs):
#   os.remove(ifile)

# files = glob(os.path.join(code_dir, 'data', 'crop_yield-data_temperature', '*.tif'))
# for ifile in tqdm(files):
#   os.remove(ifile)

In [7]:
# fin
os.environ['TCMALLOC_LARGE_ALLOC_REPORT_THRESHOLD'] = "351627116549" 
!pwd
# !python run.py process
!python run.py engineer2

/content/gdrive/My Drive/AYield/China/pycrop-yield-prediction
County: 0, State: 1, Year: 2003, Output shape: (9, 32, 512), area: 2464
County: 0, State: 1, Year: 2004, Output shape: (9, 32, 512), area: 4839
County: 0, State: 1, Year: 2005, Output shape: (9, 32, 512), area: 4960
County: 0, State: 1, Year: 2006, Output shape: (9, 32, 512), area: 5751
County: 0, State: 1, Year: 2007, Output shape: (9, 32, 512), area: 7631
County: 0, State: 1, Year: 2008, Output shape: (9, 32, 512), area: 9019
County: 0, State: 1, Year: 2009, Output shape: (9, 32, 512), area: 10443
County: 0, State: 1, Year: 2010, Output shape: (9, 32, 512), area: 13228
County: 0, State: 1, Year: 2011, Output shape: (9, 32, 512), area: 17254
County: 0, State: 1, Year: 2012, Output shape: (9, 32, 512), area: 19842
County: 0, State: 1, Year: 2013, Output shape: (9, 32, 512), area: 20886
County: 0, State: 1, Year: 2014, Output shape: (9, 32, 512), area: 22047
County: 0, State: 1, Year: 2015, Output shape: (9, 32, 512), area: 1

In [8]:
fn
!earthengine authenticate

NameError: ignored

In [ ]:
# fin

import folium
import ee
import ssl
import time
from pathlib import Path
import numpy as np
from shapely import wkt
import shapely

ee.Initialize()

import ee
import ssl
import time
from pathlib import Path
import numpy as np
import shapely
from shapely import wkt
import os

from .utils import load_clean_yield_data as load
from .utils import get_tif_files
from .. import MAJOR_STATES



In [ ]:
# fin

import folium
import ee
import ssl
import time
from pathlib import Path
import numpy as np
from shapely import wkt
import shapely

ee.Initialize()

def _export_one_image(img, folder, name, region, scale, crs):
    # export one image from Earth Engine to Google Drive
    # Author: Jiaxuan You, https://github.com/JiaxuanYou
    print(f"Exporting _exp to {folder}/{name}")
    cwd = os.getcwd()
    print('cwd : ', cwd)
    print('folder : ', folder)
    # print('_exp img : ', type(img))
    # print('_exp region : ', type(region))
    # print('_exp img : ', type(img), img.getInfo())
    # folder = "/content/gdrive/My Drive/AYield/China/pycrop-yield-prediction/data/crop_yield-data_image"
    task_dict = {
        "driveFolder": folder,
        "driveFileNamePrefix": name,
        "scale": scale,
        "crs": crs,
    }
    if region is not None:
        task_dict.update({"region": region})
    task = ee.batch.Export.image(img, name, task_dict)
    task.start()
    while task.status()["state"] == "RUNNING":
        print("Running...")
        # Perhaps task.cancel() at some point.
        time.sleep(10)
    # print('_exp img : ', type(img), img.getInfo())
    # print('_exp region : ', type(region))
    print(f"Done: {task.status()}")

def _append_mask_band(current, previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(
        ee.Algorithms.IsEqual(previous, None),
        current,
        previous.addBands(ee.Image(current)),
    )


def _append_temp_band(current, previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0, 4])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(
        ee.Algorithms.IsEqual(previous, None),
        current,
        previous.addBands(ee.Image(current)),
    )


def _append_im_band(current, previous):
    # Transforms an Image Collection with 1 band per Image into a single Image with items as bands
    # Author: Jamie Vleeshouwer

    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0, 1, 2, 3, 4, 5, 6])
    # Append it to the result (Note: only return current item on first element/iteration)
    return ee.Algorithms.If(
        ee.Algorithms.IsEqual(previous, None),
        current,
        previous.addBands(ee.Image(current)),
    )


collection_id='MODIS/006/MOD09A1'

imgcoll = (
    ee.ImageCollection(collection_id)
    .filterBounds(ee.Geometry.Rectangle(70, 54, 135, 17))
    .filterDate("2002-12-31", "2005-8-4")
    )

datatype_to_func = {
    "image": _append_im_band,
    "mask": _append_mask_band,
    "temperature": _append_temp_band,
    }

data_type = 'image'

img = imgcoll.iterate(datatype_to_func[data_type])
img = ee.Image(img)

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Load an image.
# image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')
!pwd
!ls
import sys
from cyp.data.utils import load_clean_yield_data as load
locations_filepath=Path("data/china_yield/wheat_2002-2018.csv")
locations = load(locations_filepath)

# print(type(locations))
# locations.head(5)
# sys.exit(0)
# fn

################################################################################
features=[]
for index, row in locations.iterrows():
    # polyg = i.geometry
    # print(type(row), row['geometry'])
    polyg = wkt.loads(row['geometry'])
    # print('polyg : ', index, type(polyg), polyg)
    polyg_gson = [{'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(polyg)}]
    # print(type(polyg_gson[0]['geometry']['coordinates']), list(polyg_gson[0]['geometry']['coordinates'][0]))
    tuples = list(polyg_gson[0]['geometry']['coordinates'][0])
    polyg_lists = [list(x) for x in tuples]
    # print(type(polyg_lists), polyg_lists)
    g=ee.Geometry.Polygon(polyg_lists) 
    # print('GEE 1: ', type(g), g.getInfo())
    # print('GEE 2: ', type(g), g.getInfo()['coordinates'])
    feature = ee.Feature(g, {'County ANSI':ee.String(str(row['County ANSI'])), 'State ANSI':ee.String(str(row['State ANSI']))})
    # print('Features 1: ', type(feature), feature.getInfo())
    # print('Features 2: ', feature)
    features.append(feature)
    # fin
print("Feature collection done")               
region = ee.FeatureCollection(features)
print('region : ', type(region))
# print('region : ', type(region), region.getInfo())
# fin
###############################################################################

for state_id, county_id in np.unique(
    locations[["State ANSI", "County ANSI"]].values, axis=0
):
    # if major_states_only:
    #     if int(state_id) not in MAJOR_STATES:
    #         print(f"Skipping state id {int(state_id)}")
    #         continue

    fname = "{}_{}".format(int(state_id), int(county_id))
    print('fname : ', fname)

    file_region = region.filterMetadata(
        "County ANSI", "equals", str(county_id)
    ).filterMetadata("State ANSI", "equals", str(state_id))
    # print('fr: ', type(file_region), type(file_region.first()))
    # print('fr: ', type(file_region), type(file_region.geometry().getInfo()))
    # # print('fr: ', type(file_region), file_region.geometry().getInfo())
    # print('fr: ', type(file_region), file_region.geometry().getInfo()['coordinates'])
    # file_region = ee.Feature(file_region)

    # processed_img = image.clip(file_region)
    # print(type(image), type(processed_img))

    file_region = ee.Feature(file_region.first())
    processed_img = img.clip(file_region)
    # print('processed_img.getInfo() : ', processed_img.getInfo())

    # Define the visualization parameters.
    image_viz_params = {
        # 'bands': ['B5', 'B4', 'B3'],
        'bands': ['sur_refl_b05', 'sur_refl_b04', 'sur_refl_b03'], 
        # 'min': 0,
        # 'max': 0.5,
        # 'gamma': [0.95, 1.1, 1]
    }

    coordinate_system="EPSG:4326",
    scale=500

    file_region = None
    while True:
        try:
          _export_one_image(
                processed_img,
                # download_folder,
                "/content/gdrive/My Drive/AYield/China/pycrop-yield-prediction/data/crop_yield-data_image",
                fname,
                file_region,
                scale,
                coordinate_system,
            )
        except ee.ee_exception.EEException as e:
          print(str(e))
        except ssl.SSLEOFError as e:
          print(str(e))
        # except (ee.ee_exception.EEException, ssl.SSLEOFError):
        #     print(f"Retrying State {int(state_id)}, County {int(county_id)}")
        #     print(ee.ee_exception.EEException)
        #     print(ssl.SSLEOFError)
        #     time.sleep(10)
          break
          continue
        break

    # fin



    # Define a map centered on San Francisco Bay.
    map_l8 = folium.Map(location=[35, 100], zoom_start=10)

    # Add the image layer to the map and display it.
    map_l8.add_ee_layer(processed_img, image_viz_params, 'false color composite')
    display(map_l8)
    sys.exit(0)

In [ ]:
locations.head(5)

In [ ]:
# !python run.py process
!python run.py export

In [ ]:
# fn
!earthengine authenticate